# Threaded Progress Reporting

In this example, one thread performs a long-running computation, while another thread 
runs in parallel to monitor and report progress.  

This pattern is common in scientific workflows where you want to **observe or log progress**
without interrupting the main computation — for example, during long simulations or 
model training loops.

Key ideas to notice:
- Both threads share memory, so the monitor can check the worker’s status directly.  
- Output from both threads interleaves, showing **true concurrency** within one process.  
- This is not faster, but it improves **responsiveness and usability** in real systems.


In [ ]:
import threading, time

def worker():
    for i in range(5):
        time.sleep(1)
        print(f"Work chunk {i} done")

def monitor():
    while t.is_alive():
        print("Still working...")
        time.sleep(0.5)

t = threading.Thread(target=worker)
m = threading.Thread(target=monitor)

t.start(); m.start()
t.join(); m.join()

# Coordinated Worker and Monitor Threads

In this example, several worker threads perform iterative computations, while a monitor 
thread periodically checks shared progress and reports global status.

This pattern mimics how a real scientific ML system might:
- Run parallel simulation or training steps, and  
- Use a separate monitoring thread for live progress or logging.

Key ideas:
- Workers update shared state (`progress`, `results`) under a **lock** to prevent race conditions.  
- The monitor uses a **threading.Event** to stop cleanly once all work is done.  
- Output interleaves naturally — demonstrating real concurrency without multiprocessing.


In [ ]:
import threading, time, random

# Shared state
progress = 0
results = []
lock = threading.Lock()
stop_event = threading.Event()


def worker(worker_id, n_steps=5):
    """Each worker performs n_steps of 'work' and updates global progress."""
    
    global progress, results
    
    for step in range(n_steps):
        time.sleep(random.uniform(0.3, 0.8))  # simulate computation or I/O
        result = (worker_id, step, random.random())
        
        with lock:  # safe update, atomic operation
            results.append(result)
            progress += 1
    
    print(f"Worker {worker_id} finished.")


def monitor(total_steps, interval=0.5):
    """Periodically report shared progress until stop_event is set."""
    
    while not stop_event.is_set():
        
        with lock: # safe read, atomic operation
            done = progress
    
        print(f"[Monitor] Progress: {done}/{total_steps}")
    
        if done >= total_steps:
            stop_event.set()  # signal completion
    
        time.sleep(interval)
    
    print("[Monitor] All work completed!")


# Launch several workers + one monitor
n_workers = 4
steps_per_worker = 5
total_steps = n_workers * steps_per_worker

worker_threads = [
    threading.Thread(target=worker, args=(wid, steps_per_worker))
    for wid in range(n_workers)
]
monitor_thread = threading.Thread(target=monitor, args=(total_steps,))

# Start all threads
for t in worker_threads:
    t.start()
monitor_thread.start()

# Wait for all workers
for t in worker_threads:
    t.join()
stop_event.set()  # ensure monitor stops
monitor_thread.join()

print(f"\nCollected {len(results)} results. Sample:")
print(results[:5])


[Monitor] Progress: 0/20
[Monitor] Progress: 2/20
[Monitor] Progress: 6/20
[Monitor] Progress: 10/20
[Monitor] Progress: 13/20
Worker 2 finished.
[Monitor] Progress: 17/20
Worker 3 finished.
Worker 1 finished.
[Monitor] Progress: 19/20
Worker 0 finished.
[Monitor] All work completed!

Collected 20 results. Sample:
[(3, 0, 0.1381693565263833), (2, 0, 0.07825339989324076), (1, 0, 0.1111042424182569), (0, 0, 0.4141088902810677), (2, 1, 0.33455019948234554)]
